In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import numpy as np
import pathlib
import os
import io
import cv2
import pickle
from pymongo import MongoClient

W1214 13:23:27.095091 14672 deprecation.py:323] From c:\users\asus\appdata\local\programs\python\python37\lib\site-packages\tensorflow\python\compat\v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
client = MongoClient()

In [3]:
db = client.get_database("test")

In [4]:
record = db.parking_space1

In [5]:
# list(record.find())

In [6]:
model_path = "C:/Users/asus/Desktop/Major/car_detection_model"


In [7]:
%%capture
session = tf.Session(graph=tf.Graph())
tf.saved_model.loader.load(session, ['serve'], model_path)

In [8]:
# cam = cv2.VideoCapture("./11.png")
# slots = []
# n = 100;
# while True:
#     _, im0 = cam.read()
#     showCrosshair = False
#     fromCenter = False
#     r = cv2.selectROI("Image1", im0, fromCenter, showCrosshair)
#     n = n-1
#     slots.append(r)
#     for slot in slots:
#         im0 = cv2.rectangle(im0, (int(slot[0]),int(slot[1])), 
#                                     (int(slot[0]+slot[2]),int(slot[1]+slot[3])), (255, 0, 0)) 
#     cv2.imshow("Image", im0)
    
#     if cv2.waitKey(0) & 0xFF == ord('q'):
#         break
        
# cam.release()
# cv2.destroyAllWindows()

In [9]:
def draw_boxes(height, width, box, img):
    ymin = int(max(1, (box[0] * height)))
    xmin = int(max(1, (box[1] * width)))
    ymax = int(min(height, (box[2] * height)))
    xmax = int(min(width, (box[3] * width)))
    cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (0, 0, 255), 1)

In [10]:
def predict(img_name,img):
    result = session.run(['detection_boxes:0','detection_scores:0'], feed_dict={
            'encoded_image_string_tensor:0': [img_name]})     
    boxes = result[0][0]
    scores = result[1][0]

    imH, imW, _ = img.shape
    b = []
    for i in range(len(scores)):
        if scores[i] > 0.5:
            draw_boxes(imH, imW, boxes[i], img)
            b.append(boxes[i])
    return b

In [11]:
pickle_in = open("slots_demo.pickle","rb")
slots = pickle.load(pickle_in)

In [ ]:
cam = cv2.VideoCapture("./demo4.mp4")
i=0
while True:
    ret, im0 = cam.read()
    key_pressed = cv2.waitKey(1)&0xFF
    if ret==False:
        print("Something went wrong")
        break
    if key_pressed == ord('q'):
        break
    i = i + 1
    if i%15!=0:
        continue
#     print(i)
    org = im0.copy()
    sts = {}
    for ids in slots.keys():
        if int(ids[1:]) > 70:
            continue
        slot = slots[ids]
        final = im0
        final = final*0
        x,y,w,h = slot
        final[y:y+h,x:x+w] = org[y:y+h,x:x+w]
#         cv2.imshow("slot"+str(i),final)
        
        img_str = cv2.imencode('.jpg', final)[1].tostring()
        
        found = predict(img_str,final)
        
        if found:
            sts[ids] = "occupied"
            im0 = cv2.rectangle(im0, (int(x),int(y)), 
                                    (int(x+w),int(y+h)), (0, 0, 255))
        else:
            sid = ids[1:]
            status = record.find_one({"slotID":sid})["status"]
            print(sid,status)
            if status=="alloted":
                im0 = cv2.rectangle(im0, (int(x),int(y)), 
                                    (int(x+w),int(y+h)), (255, 255, 0))
            else:
                sts[ids] = "free"
                im0 = cv2.rectangle(im0, (int(x),int(y)), 
                                    (int(x+w),int(y+h)), (0, 255, 0),2)
    for ids in sts.keys():
        sid = ids[1:]
        record.update_one({"slotID":sid},{"$set":{"status":sts[ids]}})
    
    cv2.imshow("final", im0)        
cam.release()
cv2.destroyAllWindows()

23 booked
24 free
26 alloted
27 free
28 free
30 free
48 free
49 free
54 free
60 free
61 free
68 free
70 free
23 free
24 free
26 alloted
27 free
28 free
30 free
48 free
49 free
54 free
61 free
68 free
70 free
23 free
24 free
26 alloted
27 free
28 free
30 free
48 free
49 free
54 free
61 free
68 free
70 free
23 free
24 free
26 alloted
27 free
28 free
30 free
48 free
49 free
54 free
61 free
68 free
70 free
23 free
24 free
26 alloted
27 free
28 free
30 free
48 free
49 free
54 free
61 free
68 free
70 free
23 free
24 free
26 alloted
27 free
28 free
30 free
48 free
49 free
54 free
61 free
68 free
70 free
24 free
26 alloted
27 free
28 free
30 free
48 free
49 free
54 free
61 free
68 free
70 free
23 occupied
24 free
26 alloted
27 free
28 free
30 free
48 free
49 free
54 free
61 free
68 free
70 free
24 free
26 alloted
27 free
28 free
30 free
48 free
49 free
54 free
61 free
68 free
70 free
24 free
26 alloted
27 free
28 free
30 free
48 free
49 free
54 free
60 occupied
61 free
68 free
70 free
24 free


In [ ]:
# record.update_one({"slotID":"1},{"$set":{"status":"hello"}})
# list(record.find())
# sts
record.find_one({"slotID":"26"})["status"]

In [ ]:
def predict2(img_name,img):
    result = session.run(['detection_boxes:0','detection_scores:0'], feed_dict={
            'encoded_image_string_tensor:0': [img_name]})     
    boxes = result[0][0]
    scores = result[1][0]
    
    imH, imW, _ = img.shape
    b = []
    for i in range(len(scores)):
        if scores[i] > 0.5:
            draw_boxes(imH, imW, boxes[i], img)
            b.append(boxes[i])
            
    new_img = cv2.resize(img, (1080, 1920))

    cv2.imshow("i", img)
    
    cv2.waitKey(0)

    
    

In [ ]:
img = cv2.imread("11.png")
print(img.shape)
img_str = cv2.imencode('.jpg', img)[1].tostring()
predict2(img_str,img)